# 数据预处理
在本步骤中，我们会：
1. 读取多个CSV文件并合并成一个DataFrame
2. 处理缺失值和无效值
3. 对标签进行编码
4. 将数据标准化
5. 将数据分为训练集和测试集


In [ ]:
# 导入必要的库
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

### 导入库
在本单元格中，我们导入了需要的库：
- `os`：用于文件路径管理
- `pandas`：用于数据操作和处理，例如加载和合并CSV文件
- `StandardScaler` 和 `LabelEncoder`：用于数据标准化和标签编码
- `train_test_split`：用于将数据分割成训练集和测试集

**示例：**
如果我们有一个CSV文件，可以通过 `pd.read_csv('file.csv')` 来读取数据，并用 `LabelEncoder` 对目标列进行编码。


In [ ]:
# 读取和合并CSV文件
def load_and_merge_csv(folder_path):
    # 获取文件夹中所有CSV文件的路径
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    combined_data = pd.DataFrame()  # 初始化一个空的DataFrame用于存放合并的数据

    # 遍历每个CSV文件并读取数据
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        data = pd.read_csv(file_path)
        data.columns = data.columns.str.strip()  # 去除列名中的空格
        combined_data = pd.concat([combined_data, data], ignore_index=True)  # 合并到总的DataFrame中
    return combined_data


### 读取和合并CSV文件
该函数用于读取指定文件夹下所有CSV文件，并将它们合并为一个`DataFrame`。
- `folder_path`：指定的文件夹路径。
- `os.listdir(folder_path)`：获取文件夹中所有文件名。
- `pd.read_csv(file_path)`：读取CSV文件。
- `pd.concat()`：将多个`DataFrame`合并为一个。


In [ ]:
# 处理缺失值
def handle_missing_values(data):
    # 仅处理数值列的缺失值，填充为中位数
    numeric_columns = data.select_dtypes(include=['number']).columns
    data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].median())
    return data


### 处理缺失值
在这个函数中，我们处理数据中的缺失值，特别是数值列的缺失值。
- `select_dtypes(include=['number'])`：选取数据中的数值列。
- `fillna(median)`：用每列的中位数来填充缺失值，以减少异常值的影响。


In [ ]:
# 处理无效值（无穷大值和NaN）
def handle_invalid_values(data):
    data = data.replace([float('inf'), float('-inf')], float('nan'))  # 替换无穷大为NaN
    data = data.fillna(0)  # 填充NaN值为0
    return data


### 处理无效值
此函数将数据中的无效值（无穷大值）替换为 `NaN`，并进一步用 `0` 填充所有 `NaN` 值。
- `replace([float('inf'), float('-inf')], float('nan'))`：替换正无穷和负无穷为 `NaN`。
- `fillna(0)`：填充所有的 `NaN` 值为 `0`。

In [ ]:
# 标签编码（将分类标签转换为数字）
def encode_labels(data, label_column='Label'):
    if label_column in data.columns:
        label_encoder = LabelEncoder()
        data[label_column] = label_encoder.fit_transform(data[label_column])  # 编码标签列
        return data, label_encoder
    else:
        return data, None  # 如果没有指定的标签列，返回原数据


### 标签编码
该函数用于对数据中的目标列（标签列）进行编码，以便将非数值的类别标签转换为数值表示，这样模型可以直接使用。
- `label_column`：指定标签列的名称，默认值为 `Label`。
- `LabelEncoder()`：用于将类别标签转化为数值。
- `fit_transform()`：直接将标签列的内容进行编码并替换。

In [ ]:
# 特征标准化
def scale_features(data, feature_columns):
    scaler = StandardScaler()
    data[feature_columns] = scaler.fit_transform(data[feature_columns])  # 将特征缩放为标准正态分布
    return data, scaler


### 特征标准化
此函数用于对数据集的特征列进行标准化处理，将数据转换为零均值和单位方差，以保证不同特征的数值尺度一致。
- `StandardScaler()`：对特征进行标准化处理。
- `fit_transform()`：对数据进行拟合和转换，将数值标准化。

In [ ]:
# 数据分割，将数据划分为训练集和测试集
def split_data(data, label_column='Label', test_size=0.2):
    X = data.drop(columns=[label_column])  # 特征集
    y = data[label_column]  # 标签列
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    return X_train, X_test, y_train, y_test


### 数据集划分
该函数将数据集划分为训练集和测试集，分别用于模型的训练和验证。
- `drop(columns=[label_column])`：将标签列（目标变量）从数据中移除，剩下的列作为特征。
- `train_test_split()`：以 `80:20` 的比例（默认）划分数据集。

In [ ]:
# 运行数据预处理流程
folder_path = '../data/CICIDS2017/'
output_dir = '../data/processed'
combined_data = load_and_merge_csv(folder_path)  # 加载并合并CSV文件
combined_data = handle_missing_values(combined_data)  # 处理缺失值
combined_data = handle_invalid_values(combined_data)  # 处理无效值
feature_columns = combined_data.select_dtypes(include=['number']).columns  # 选择数值列进行标准化
combined_data, label_encoder = encode_labels(combined_data)  # 编码标签列

if label_encoder is not None:
    combined_data, scaler = scale_features(combined_data, feature_columns)  # 特征标准化
    X_train, X_test, y_train, y_test = split_data(combined_data)  # 数据集划分

    # 保存处理后的数据
    os.makedirs(output_dir, exist_ok=True)
    X_train.to_csv(f'{output_dir}/X_train.csv', index=False)
    y_train.to_csv(f'{output_dir}/y_train.csv', index=False)
    X_test.to_csv(f'{output_dir}/X_test.csv', index=False)
    y_test.to_csv(f'{output_dir}/y_test.csv', index=False)
    print("数据预处理完成并保存！")
else:
    print("由于缺少 'Label' 列，跳过预处理。")


### 数据预处理流程说明

此单元格执行数据预处理的主要步骤，以下是具体操作说明：

1. **指定文件路径**：定义数据文件夹 (`folder_path`) 和保存预处理结果的文件夹 (`output_dir`) 路径。
2. **加载并合并CSV文件**：调用 `load_and_merge_csv` 函数，读取并合并文件夹中的CSV数据文件。
   - **示例**：`combined_data` 将包含所有CSV文件合并后的数据。
3. **处理缺失值**：调用 `handle_missing_values` 函数，填补数据中的缺失值。
4. **处理无效值**：使用 `handle_invalid_values` 函数，将无效数据替换为适当的默认值（如 `0` 或均值）。
5. **选择数值列进行标准化**：提取数据中的数值列名，后续步骤中将对这些列进行标准化处理。
6. **编码标签列**：使用 `encode_labels` 函数对标签列（`Label` 列）进行编码。
   - **重要**：检查标签列是否存在，如果不存在则跳过后续步骤。
7. **特征标准化**：使用 `scale_features` 函数，对数值列进行标准化处理，使其均值为0，方差为1。
8. **数据集划分**：使用 `split_data` 函数将数据集划分为训练集和测试集。
9. **保存处理后的数据**：将预处理后的训练集和测试集特征、标签数据保存到指定目录中。
   - 数据文件包括 `X_train.csv`, `y_train.csv`, `X_test.csv`, `y_test.csv`。

**示例输出**：
执行完此单元格后，处理后的数据文件会保存在 `output_dir` 目录中。输出包括处理成功信息“数据预处理完成并保存！”或缺少标签列时的提示信息。

这个单元格完成了整个数据预处理的过程，为后续模型训练和评估提供了清理和准备好的数据集。
